In [1]:
import random
import datetime

# -----------------------------
# Node class for patients
# -----------------------------
class BinomialHeapNode:
    def __init__(self, patient_id, severity, waiting_time, age, special_needs, check_in_time, appointment_id):
        self.patient_id = patient_id
        self.severity = severity
        self.waiting_time = waiting_time
        self.age = age
        self.special_needs = special_needs
        self.check_in_time = check_in_time
        self.appointment_id = appointment_id

        self.degree = 0
        self.parent = None
        self.child = None
        self.sibling = None

        self.priority_score = self.compute_priority()

    def compute_priority(self):
        # Severity weights
        severity_weight = {"Emergency": 100, "Critical": 50, "Urgent": 20, "Normal": 0}[self.severity]
        # Age factor: seniors get more weight
        if self.age < 12:
            age_factor = 5
        elif self.age < 60:
            age_factor = self.age / 10
        else:
            age_factor = self.age / 8
        # Special needs bonus
        special_bonus = 10 if self.special_needs == "Yes" else 0
        # Waiting time
        waiting_bonus = 0.6 * self.waiting_time

        return severity_weight + age_factor + special_bonus + waiting_bonus

    def __lt__(self, other):
        # higher priority_score = higher urgency, so min-heap uses negative
        if self.priority_score != other.priority_score:
            return self.priority_score > other.priority_score  # higher score first
        # tie-breaker rules
        if self.severity != other.severity:
            severity_order = {"Emergency":4,"Critical":3,"Urgent":2,"Normal":1}
            return severity_order[self.severity] > severity_order[other.severity]
        if self.special_needs != other.special_needs:
            return self.special_needs == "Yes"
        if self.check_in_time != other.check_in_time:
            return self.check_in_time < other.check_in_time
        if self.age != other.age:
            # pick child or senior first (policy: child <12, senior >60)
            return (self.age < 12 or self.age > 60)
        return self.appointment_id < other.appointment_id  # final deterministic tie-breaker

# -----------------------------
# Binomial Heap class
# -----------------------------
class BinomialHeap:
    def __init__(self):
        self.head = None

    def merge_root_lists(self, h1, h2):
        if not h1: return h2
        if not h2: return h1

        if h1.degree <= h2.degree:
            head = h1
            h1 = h1.sibling
        else:
            head = h2
            h2 = h2.sibling

        tail = head
        while h1 and h2:
            if h1.degree <= h2.degree:
                tail.sibling = h1
                h1 = h1.sibling
            else:
                tail.sibling = h2
                h2 = h2.sibling
            tail = tail.sibling

        tail.sibling = h1 if h1 else h2
        return head

    def link_trees(self, lesser, greater):
        greater.parent = lesser
        greater.sibling = lesser.child
        lesser.child = greater
        lesser.degree += 1

    def union(self, other_heap):
        new_head = self.merge_root_lists(self.head, other_heap.head)
        if not new_head:
            return
        prev = None
        curr = new_head
        nxt = curr.sibling

        while nxt:
            if (curr.degree != nxt.degree) or (nxt.sibling and nxt.sibling.degree == curr.degree):
                prev = curr
                curr = nxt
            else:
                if curr < nxt:
                    curr.sibling = nxt.sibling
                    self.link_trees(curr, nxt)
                else:
                    if prev:
                        prev.sibling = nxt
                    else:
                        new_head = nxt
                    self.link_trees(nxt, curr)
                    curr = nxt
            nxt = curr.sibling

        self.head = new_head

    def insert(self, node: BinomialHeapNode):
        new_heap = BinomialHeap()
        new_heap.head = node
        self.union(new_heap)

    def extract_max(self):
        if not self.head:
            return None

        max_prev = None
        max_node = self.head
        prev = self.head
        curr = prev.sibling

        while curr:
            if curr < max_node:
                max_node = curr
                max_prev = prev
            prev = curr
            curr = curr.sibling

        if max_prev:
            max_prev.sibling = max_node.sibling
        else:
            self.head = max_node.sibling

        # Reverse children and union back
        child = max_node.child
        prev = None
        while child:
            nxt = child.sibling
            child.sibling = prev
            child.parent = None
            prev = child
            child = nxt

        new_heap = BinomialHeap()
        new_heap.head = prev
        self.union(new_heap)

        return max_node

# -----------------------------
# DEMO
# -----------------------------
if __name__ == "__main__":
    bh = BinomialHeap()

    # Generate a few patients manually
    patients = [
        BinomialHeapNode("P001", "Urgent", 30, 30, "No", datetime.datetime(2025,9,25,9,0), 1),
        BinomialHeapNode("P002", "Emergency", 10, 5, "No", datetime.datetime(2025,9,25,9,0), 2),
        BinomialHeapNode("P003", "Critical", 20, 65, "Yes", datetime.datetime(2025,9,25,9,30), 3),
        BinomialHeapNode("P004", "Normal", 15, 40, "No", datetime.datetime(2025,9,25,8,50), 4),
        BinomialHeapNode("P005", "Urgent", 60, 2, "Yes", datetime.datetime(2025,9,25,9,10), 5),
    ]

    for p in patients:
        bh.insert(p)

    print("Serving order (Patient ID, Priority Score, Age, Severity):")
    while True:
        served = bh.extract_max()
        if not served:
            break
        print(served.patient_id, round(served.priority_score,2), served.age, served.severity)


Serving order (Patient ID, Priority Score, Age, Severity):
P002 111.0 5 Emergency
P003 80.12 65 Critical
P005 71.0 2 Urgent
P001 41.0 30 Urgent
P004 13.0 40 Normal
